In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [3]:
%cd drive/Shareddrives/CS704_Project/data_processing

/content/drive/Shareddrives/CS704_Project/data_processing


In [4]:
import pandas as pd
df = pd.read_csv("processed_data_3_majority.csv")
df.head()

<ipython-input-4-8060f93470f0>:2: DtypeWarning: Columns (0,1,2,3,4,5,11,12,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("processed_data_3_majority.csv")


,tweet,target,manual_keywords,statement,3_label_majority_answer,followers_count,friends_count,favourites_count,statuses_count,listed_count,...,dots,exclamation,questions,ampersand,capitals,digits,long_word_freq,short_word_freq,cleaned_tweet,Truthfulness
0,@POTUS Biden Blunders - 6 Month Update\n\nInfl...,True,"Americans, eviction moratorium",End of eviction moratorium means millions of A...,Agree,4262.0,3619.0,34945.0,16423.0,44.0,...,5.0,0.0,1.0,0.0,33.0,3.0,5.0,19.0,Biden Blunders 6 Month Update Inflation Delt...,True
1,@S0SickRick @Stairmaster_ @6d6f636869 Not as m...,True,"Americans, eviction moratorium",End of eviction moratorium means millions of A...,Agree,1393.0,1621.0,31436.0,37184.0,64.0,...,1.0,0.0,0.0,0.0,14.0,0.0,2.0,34.0,Not as many people are literally starving and ...,True
2,THE SUPREME COURT is siding with super rich pr...,True,"Americans, eviction moratorium",End of eviction moratorium means millions of A...,Agree,9.0,84.0,219.0,1184.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,4.0,10.0,THE SUPREME COURT is siding with super rich pr...,True
3,@POTUS Biden Blunders\n\nBroken campaign promi...,True,"Americans, eviction moratorium",End of eviction moratorium means millions of A...,Agree,4262.0,3619.0,34945.0,16423.0,44.0,...,3.0,0.0,0.0,1.0,6.0,8.0,1.0,30.0,Biden Blunders Broken campaign promises Infla...,True
4,@OhComfy I agree. The confluence of events rig...,True,"Americans, eviction moratorium",End of eviction moratorium means millions of A...,Agree,70.0,166.0,15282.0,2194.0,0.0,...,3.0,0.0,1.0,0.0,11.0,3.0,2.0,19.0,I agree The confluence of events right now is ...,True


In [5]:
df = df[df['Truthfulness'].isin([True, False])]

In [6]:
X = df.select_dtypes(include='number').assign(cleaned_tweet=df['cleaned_tweet'])
y = df['Truthfulness']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
X_train_1 = X_train.drop('cleaned_tweet', axis=1)
y_train_1 = y_train.map({True: 1, False: 0})

X_val_1 = X_temp.drop('cleaned_tweet', axis=1)
y_val_numeric = y_temp.map({True: 1, False: 0})

(92982, 56)

In [ ]:
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict, cross_val_score, KFold, cross_validate
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

n_estimators_list = list(range(100, 1001, 150))
max_features_list = ['sqrt', 0.3]
min_samples_split = 2
max_depth = None

for n_estimators in n_estimators_list:
    for max_features in max_features_list:
        print("n_estimators: {} --- max_features: {}".format(n_estimators, max_features))
        scoring = {'accuracy' : make_scorer(accuracy_score),
                  'precision' : make_scorer(precision_score),
                  'recall' : make_scorer(recall_score),
                  'f1_score' : make_scorer(f1_score)}

        kfold = KFold(n_splits=10)
        model=RandomForestClassifier(n_estimators=n_estimators, max_features=max_features, min_samples_split=min_samples_split,
                  max_depth=max_depth,
                  random_state=42)

        results = cross_validate(estimator=model,
                                  X=X_train_1,
                                  y=y_train_1,
                                  cv=kfold,
                                  scoring=scoring)
        print(results['test_accuracy'].mean())
        print(results['test_f1_score'].mean())
        print(results['test_precision'].mean())
        print(results['test_recall'].mean())
        print("\n" + "="*50 + "\n")

n_estimators: 100 --- max_features: sqrt
0.6692154280496211
0.6663588654206981
0.6843800885701057
0.649350032915903


n_estimators: 100 --- max_features: 0.3
0.6723988493825885
0.6675864224424731
0.6899949448325364
0.6466676317797544


n_estimators: 250 --- max_features: sqrt
0.6757973730502179
0.6745519995377665
0.6893182668266183
0.6604941534914344


n_estimators: 250 --- max_features: 0.3
0.6768405445428565
0.6734708604389155
0.6929813709446311
0.6551244650434098


n_estimators: 400 --- max_features: sqrt
0.6776579084325292
0.6764964245920606
0.6911544326853983
0.6625381083846222


n_estimators: 400 --- max_features: 0.3
0.6782279338987156
0.6749566901850962
0.6943222117059495
0.6567508286022627


n_estimators: 550 --- max_features: sqrt
0.6778622522963876
0.6769943874589803
0.6910272093471992
0.6636171106618883


n_estimators: 550 --- max_features: 0.3
0.6791420488481762
0.6761871663287191
0.6948923188619937
0.6585477407427576


n_estimators: 700 --- max_features: sqrt
0.6779912891

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-338176ff2eeb>", line 24, in <cell line: 11>
    results = cross_validate(estimator=model,
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 266, in cross_validate
    results = parallel(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py", line 63, in __call__
    return super().__call__(iterable_with_config)
  File "/usr/local/lib/python3.10/dist-packages/joblib/parallel.py", line 1863, in __call__
    return output if self.return_generator else list(output)
  File "/usr/local/lib/python3.10/dist-packages/joblib/parallel.py", line 1792, in _get_sequential_output
    res = func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py", line 123, in __call__
    return s